In [1]:
(
    1 + 0
)  # Utility cell. Sometimes "Jupyter" skipps the first cell when running all cells. Especially after a restart.

1

In [2]:
(
    1 + 1
)  # Utility cell. Sometimes "Jupyter" skipps the first cell when running all cells. Especially after a restart.

2

In [3]:
print(f"N{1+2} utility cell running...")

N3 utility cell running...


In [4]:
# Resolve the path to the 'src' folder with helper functions
import config_notebooks

PYTHONPATH: C:\Users\Admin\Documents\Univer\MASTER THESIS IDEAS\Flavio Pinehro\11_MasterProject\src


In [5]:
from n03_import import *

os.getcwd():
c:\Users\Admin\Documents\Univer\MASTER THESIS IDEAS\Flavio Pinehro\11_MasterProject\notebooks
input_file_location:
..\data\02_ingested
output_file_location:
..\data\03_preprocessed


In [6]:
# # Resolve the path to the 'src' folder with helper functions
# import config_notebooks
# from importlib import reload

# # Import the module and reload it
# import n03_import_reload

# reload(n03_import_reload)

# # Re-import the functions from the reloaded module
# from n03_import_reload import *

## Processing 'GDP per capita' data

In [7]:
# Raw gdp per capita data
categorical_columns = ["country_code", "year"]
input_file_name = "ingested_gdp_pcap.dta"
gdp_pcap_df = read_hs92_stata_data(
    input_file_location, input_file_name, categorical_columns=categorical_columns
)

country_code column initial type:
object
country_code column final type:
category
year column initial type:
int16
year column final type:
category


In [8]:
gdp_pcap_df.head()

,country_code,year,gdp_per_capita,log_gdp_per_capita
0,AUS,1960,"1,810.71",7.50
1,AUT,1960,939.91,6.85
2,BDI,1960,70.91,4.26
3,BEL,1960,"1,290.29",7.16
4,BEN,1960,89.86,4.50


In [9]:
df_stats(gdp_pcap_df)

['df', <class 'pandas.core.frame.DataFrame'>]
shape:
(8334, 4)
data types:
country_code          category
year                  category
gdp_per_capita         float64
log_gdp_per_capita     float64
dtype: object
columns:
['country_code', 'year', 'gdp_per_capita', 'log_gdp_per_capita']
       gdp_per_capita  log_gdp_per_capita
count        8,334.00            8,334.00
mean         7,104.50                7.49
std         13,417.95                1.73
min             11.80                2.47
25%            462.09                6.14
50%          1,554.43                7.35
75%          6,321.94                8.75
max        108,798.45               11.60
Total memory usage: 173110 bytes


### Preprocessing GDP data

In [10]:
# this data was staged at n03_Processed_exports.ipynb
dumped_data = restore_cache(input_file_location=input_file_location)

data restored:['valid_gdp_country_codes', 'valid_gdp_country_ids', 'valid_gdp_locations', 'export_years', 'unreliable_locations', 'rolling_exports_years']


In [11]:
# # Better to switch to string data type for categorical columns before processing
# categorical_columns = ["year"]
# gdp_pcap_df = switch_to_dtype(
#     gdp_pcap_df, default_dtype="str", categorical_columns=categorical_columns
# )

### Subsetting first to the whole range of years under observation in order to select countries with consistent data

In [12]:
export_years = dumped_data["export_years"]
# export_years = [*map(str, export_years)]
export_years

{2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019}

In [13]:
gdp_pcap_df = gdp_pcap_df[gdp_pcap_df["year"].isin(export_years)]

In [14]:
# drop unused categories in the categorical "year" column
# Check categories before and after
print("Before:", gdp_pcap_df["year"].cat.categories, "\n")
gdp_pcap_df["year"] = (
    gdp_pcap_df["year"].astype("category").cat.remove_unused_categories()
)
print("After:", gdp_pcap_df["year"].cat.categories)

Before: Index([1960, 1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971,
       1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983,
       1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995,
       1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007,
       2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019,
       2020, 2021, 2022, 2023],
      dtype='int16') 

After: Index([2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019], dtype='int16')


In [15]:
df_stats(gdp_pcap_df, "gdp_pcap_df")

['gdp_pcap_df', <class 'pandas.core.frame.DataFrame'>]
shape:
(1483, 4)
data types:
country_code          category
year                  category
gdp_per_capita         float64
log_gdp_per_capita     float64
dtype: object
columns:
['country_code', 'year', 'gdp_per_capita', 'log_gdp_per_capita']
       gdp_per_capita  log_gdp_per_capita
count        1,483.00            1,483.00
mean        13,660.96                8.56
std         19,089.61                1.47
min            210.24                5.35
25%          1,492.60                7.31
50%          5,259.31                8.57
75%         15,689.09                9.66
max        108,470.38               11.59
Total memory usage: 53229 bytes


In [16]:
# Searching for the missing values in the merged dataframe in the 'gdp_per_capita' column
missing_gdp_pcap_df = gdp_pcap_df[gdp_pcap_df["gdp_per_capita"].isnull()]
missing_gdp_pcap_df

,country_code,year,gdp_per_capita,log_gdp_per_capita


No missing values may be confusing, first because the data is in long format, and second because we know that data is missing at least for "VEN" which is Venezuela

In [17]:
missing_gdp_pcap_df = gdp_pcap_df[gdp_pcap_df["country_code"] == "VEN"]
missing_gdp_pcap_df

,country_code,year,gdp_per_capita,log_gdp_per_capita
6412,VEN,2010,"13,646.30",9.52
6562,VEN,2011,"10,843.92",9.29
6711,VEN,2012,"12,901.42",9.47
6860,VEN,2013,"12,403.15",9.43
7009,VEN,2014,"15,943.61",9.68


In [18]:
# Count the number of unique years each country appears in
country_year_counts = gdp_pcap_df.groupby("country_code", observed=False)[
    "year"
].nunique()

In [19]:
# Identify countries that do not appear in all years
missing_years_country_codes = country_year_counts[
    country_year_counts < len(export_years)
].index

In [20]:
missing_years_country_codes = set(
    gdp_pcap_df[gdp_pcap_df["country_code"].isin(missing_years_country_codes)][
        "country_code"
    ].unique()
)
missing_years_country_codes

{'ERI', 'SSD', 'VEN'}

In [21]:
# Remove these countries from the dataset. Reset index to optimize memory layout. Works very quickly for categories instead of objects
gdp_pcap_df = gdp_pcap_df[
    ~gdp_pcap_df["country_code"].isin(missing_years_country_codes)
].reset_index(drop=True)

In [22]:
gdp_pcap_df[gdp_pcap_df["country_code"].isin(missing_years_country_codes)]

,country_code,year,gdp_per_capita,log_gdp_per_capita


In [23]:
# Drop unused categories in the categorical "country_code" column
# Check categories before and after
print("Before:", gdp_pcap_df["country_code"].cat.categories, "\n")
gdp_pcap_df["country_code"] = (
    gdp_pcap_df["country_code"].astype("category").cat.remove_unused_categories()
)
print("After:", gdp_pcap_df["country_code"].cat.categories)

Before: Index(['AFG', 'AGO', 'ALB', 'ARE', 'ARG', 'ARM', 'AUS', 'AUT', 'AZE', 'BDI',
       ...
       'UKR', 'URY', 'USA', 'UZB', 'VEN', 'VNM', 'YEM', 'ZAF', 'ZMB', 'ZWE'],
      dtype='object', length=150) 

After: Index(['AFG', 'AGO', 'ALB', 'ARE', 'ARG', 'ARM', 'AUS', 'AUT', 'AZE', 'BDI',
       ...
       'UGA', 'UKR', 'URY', 'USA', 'UZB', 'VNM', 'YEM', 'ZAF', 'ZMB', 'ZWE'],
      dtype='object', length=147)


In [24]:
gdp_pcap_df.sample(5)

,country_code,year,gdp_per_capita,log_gdp_per_capita
60,IRN,2010,"6,291.19",8.75
827,MWI,2015,539.61,6.29
1411,MNG,2019,"4,347.55",8.38
1362,ESP,2019,"29,776.20",10.30
1319,YEM,2018,633.89,6.45


### Subsetting to the years for which the average exports were calculated using 'rolling window'

In [25]:
rolling_export_years = dumped_data["rolling_exports_years"]
# rolling_export_years = [*map(str, rolling_export_years)]
rolling_export_years

{2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019}

In [26]:
gdp_pcap_df = gdp_pcap_df[gdp_pcap_df["year"].isin(rolling_export_years)]

In [27]:
df_stats(gdp_pcap_df)

['df', <class 'pandas.core.frame.DataFrame'>]
shape:
(1176, 4)
data types:
country_code          category
year                  category
gdp_per_capita         float64
log_gdp_per_capita     float64
dtype: object
columns:
['country_code', 'year', 'gdp_per_capita', 'log_gdp_per_capita']
       gdp_per_capita  log_gdp_per_capita
count        1,176.00            1,176.00
mean        13,809.46                8.59
std         19,133.48                1.46
min            210.24                5.35
25%          1,554.74                7.35
50%          5,419.38                8.60
75%         16,449.95                9.71
max        108,470.38               11.59
Total memory usage: 40796 bytes


### Looking for missing GDP values

In [28]:
# Searching for the missing values in the merged dataframe in the 'gdp_per_capita' column
missing_gdp_pcap_df = gdp_pcap_df[gdp_pcap_df["gdp_per_capita"].isnull()]

In [29]:
missing_gdp_pcap_df

,country_code,year,gdp_per_capita,log_gdp_per_capita


In [30]:
set(missing_gdp_pcap_df["country_code"].unique())

set()

In [31]:
# Dropping the missing values in the 'gdp_per_capita' column
gdp_pcap_df = gdp_pcap_df.dropna(subset=["gdp_per_capita"]).reset_index(drop=True)

In [32]:
# # Better to switch to categorical data type for categorical columns before staging
# categorical_columns = ["year"]
# gdp_pcap_df = switch_to_dtype(
#     gdp_pcap_df, default_dtype="category", categorical_columns=categorical_columns
# )

In [33]:
df_stats(gdp_pcap_df)

['df', <class 'pandas.core.frame.DataFrame'>]
shape:
(1176, 4)
data types:
country_code          category
year                  category
gdp_per_capita         float64
log_gdp_per_capita     float64
dtype: object
columns:
['country_code', 'year', 'gdp_per_capita', 'log_gdp_per_capita']
       gdp_per_capita  log_gdp_per_capita
count        1,176.00            1,176.00
mean        13,809.46                8.59
std         19,133.48                1.46
min            210.24                5.35
25%          1,554.74                7.35
50%          5,419.38                8.60
75%         16,449.95                9.71
max        108,470.38               11.59
Total memory usage: 31520 bytes


In [34]:
# Calculating log of 'gdp_per_capita' column
gdp_pcap_df["log_gdp_per_capita"] = np.log(gdp_pcap_df["gdp_per_capita"])

In [35]:
gdp_pcap_df.sample(5)

,country_code,year,gdp_per_capita,log_gdp_per_capita
472,CUB,2015,"7,727.92",8.95
813,LSO,2017,"1,069.37",6.97
17,BOL,2012,"2,575.60",7.85
1111,MDA,2019,"4,405.33",8.39
64,JAM,2012,"5,340.83",8.58


### Staging GDP data

In [36]:
# Save the clean gdp DataFrame to a .parquet file
output_file_name = "GDP.parquet"
dtype_dict = {"country_code": "category"}
save_hs92_parquet_data(
    data=gdp_pcap_df,
    file_location=output_file_location,
    file_name=output_file_name,
    dtype_dict=dtype_dict,
)

year column initial type:
category
year column final type:
int16
country_code column initial type:
category
country_code column final type:
category


In [37]:
dumped_data["reliable_gdp_codes"] = set(gdp_pcap_df["country_code"].unique())

staging_cache(output_file_location=input_file_location, **dumped_data)

data staged:['valid_gdp_country_codes', 'valid_gdp_country_ids', 'valid_gdp_locations', 'export_years', 'unreliable_locations', 'rolling_exports_years', 'reliable_gdp_codes']
